<a href="https://colab.research.google.com/github/dotsnangles/Transformers-Text-Classification/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install -q folium
!pip install -q transformers
!pip install -q datasets
!pip install -q huggingface_hub

     |████████████████████████████████| 4.2 MB 6.8 MB/s 
     |████████████████████████████████| 6.6 MB 64.5 MB/s 
     |████████████████████████████████| 86 kB 5.3 MB/s 
     |████████████████████████████████| 596 kB 76.2 MB/s 
     |████████████████████████████████| 346 kB 7.4 MB/s 
     |████████████████████████████████| 86 kB 6.5 MB/s 
     |████████████████████████████████| 1.1 MB 75.0 MB/s 
     |████████████████████████████████| 212 kB 95.9 MB/s 
     |████████████████████████████████| 140 kB 99.4 MB/s 
     |████████████████████████████████| 127 kB 87.2 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
     |████████████████████████████████| 271 kB 97.6 MB/s 
     |████████████████████████████████| 144 kB 77.4 MB/s 
     |████████████████████████████████| 112 kB 100.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 r

In [5]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
import datasets

from huggingface_hub import notebook_login

import torch

from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold

import pandas as pd
import numpy as np

import os
import gc

### Download Models and Tokenizers

In [6]:
model_names = ['klue/roberta-base', 'bert-base-multilingual-uncased', 'xlm-roberta-base']

for model_name in model_names:
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=7)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    model_name = model_name.replace('/', '-')

    model.save_pretrained(f'models/{model_name}')
    tokenizer.save_pretrained(f'models/{model_name}')

    model_name = model_name.replace('klue-roberta-base', 'klue/roberta-base')

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classif

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

### Global Configration

In [21]:
num_labels = 7
models = ['klue-roberta-base', 'bert-base-multilingual-uncased', 'xlm-roberta-base']
batch_size = 64
num_train_epochs = 3

data_path = '/content/revised_data.csv'
gdrive_path = '/content/drive/MyDrive/Data Science/알파코 딥러닝 부트캠프/프로젝트/뉴스 토픽 분류 AI 경진대회/'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Check Data

In [22]:
df = pd.read_csv(data_path, index_col=False)

In [23]:
df

,Unnamed: 0,title,topic_idx
0,0,"훌륭한 이름도 없고, 훌륭한 이름도 없고, 훌륭한 이름도 없다.",3
1,1,내일날씨 전국 구름 많음…아침 기온 다소 올라,3
2,2,김연경 금환...이제 도쿄 올림픽 출발을 위해 완성,5
3,3,브라질 최대정당 호세프와 결별할 듯…탄핵 가능성 커져,4
4,4,책·TV·PC 보면서도 스마트폰 손에서 못 뗀다,3
...,...,...,...
101859,101859,표면을 보다 고르게…신개념 정밀 코팅기술 개발,0
101860,101860,월드컵 러시아 팬도 손흥민 보러왔어요…인기 스타는 ...,5
101861,101861,레알 바르사와 함께 승리...강렬한 싸움,5
101862,101862,프로 농구 인삼공사 KCC에 연장 혈투 승리...콜페퍼 27점,5


### Function Declaration

In [24]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [25]:
def tokenize(batch):
    return tokenizer(batch["title"], padding=True, truncation=True)

### Fine Tuning and Model Saving

In [26]:
for model_n in models:
    # Load Model & Tokenizer
    model_ckpt = model_n

    # Load Data & Prepare K-Fold
    df = pd.read_csv(data_path, index_col=False)
    # df = df.drop(['Unnamed: 0'], axis=1)
    # df.reset_index(inplace=True)
    X, y = df['title'], df['topic_idx']
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=100)
    splitgen = skf.split(X, y)

    print()
    print('----------------------------------------------------------------------------')
    print(model_ckpt)

    foldNum = 1
    for train_index, test_index in splitgen:
        print()
        print('----------------------------------------------------------------------------')
        print(f'{model_ckpt} {foldNum}-Fold')

        # Model instantiation
        model = AutoModelForSequenceClassification.from_pretrained('models/' + model_ckpt, num_labels=num_labels, local_files_only=True)
        tokenizer = AutoTokenizer.from_pretrained('models/' + model_ckpt)
        data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

        # Tokenization
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        train = {
            'title': X_train.to_list(),
            'topic_idx': y_train.to_list()
        }
        
        test =  {
            'title': X_test.to_list(),
            'topic_idx': y_test.to_list()
        }
        
        train = datasets.Dataset.from_dict(train)
        train_encoded = train.map(tokenize, batched=True, batch_size=None)
        train_encoded = train_encoded.rename_column('topic_idx', 'label')

        test = datasets.Dataset.from_dict(test)
        test_encoded = test.map(tokenize, batched=True, batch_size=None)
        test_encoded = test_encoded.rename_column('topic_idx', 'label')

        # Training 
        logging_steps = len(train_encoded) // batch_size
        
        # model_ckpt = model_ckpt.replace('/', '-')
        model_name = f"{model_ckpt}-tc-{foldNum}"

        training_args = TrainingArguments(output_dir='log/' + model_name,
                                            num_train_epochs=num_train_epochs,
                                            learning_rate=2e-5,
                                            per_device_train_batch_size=batch_size,
                                            per_device_eval_batch_size=batch_size,
                                            weight_decay=0.01,
                                            evaluation_strategy="epoch",
                                            disable_tqdm=False,
                                            logging_steps=logging_steps,
                                            save_strategy='no',
                                            push_to_hub=False,
                                            log_level="error")

        trainer = Trainer(model=model,
                    args=training_args,
                    compute_metrics=compute_metrics,
                    train_dataset=train_encoded,
                    eval_dataset=test_encoded,
                    tokenizer=tokenizer,
                    data_collator=data_collator)
        
        trainer.train()

        tokenizer.save_pretrained(f'{gdrive_path}saved/{model_name}')
        model.save_pretrained(f'{gdrive_path}saved/{model_name}')

        # model_ckpt = model_ckpt.replace('klue-roberta-base', 'klue/roberta-base')

        del model
        torch.cuda.empty_cache()

        foldNum += 1


----------------------------------------------------------------------------
klue-roberta-base

----------------------------------------------------------------------------
klue-roberta-base 1-Fold


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.482600,0.369669,0.874098,0.873061
2,0.291900,0.326083,0.892799,0.891651
3,0.201900,0.313617,0.902469,0.901767


...


----------------------------------------------------------------------------
xlm-roberta-base 5-Fold


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.596100,0.410798,0.863293,0.861906
2,0.378700,0.370779,0.877921,0.876594
3,0.300200,0.353482,0.885382,0.884368


### Free GPU Memory & Cache

In [ ]:
# %cd ~/.cache/huggingface/transformers
# !ls

In [ ]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# !nvidia-smi -L

# import gc

# model.cpu()
# del model, checkpoint
# gc.collect()
# torch.cuda.empty_cache()

### Logit Ensemble

In [27]:
saved_models = os.listdir(f'{gdrive_path}saved')
saved_models.sort()

In [28]:
for e in saved_models:
    print(e)

bert-base-multilingual-uncased-tc-1
bert-base-multilingual-uncased-tc-2
bert-base-multilingual-uncased-tc-3
bert-base-multilingual-uncased-tc-4
bert-base-multilingual-uncased-tc-5
klue-roberta-base-tc-1
klue-roberta-base-tc-2
klue-roberta-base-tc-3
klue-roberta-base-tc-4
klue-roberta-base-tc-5
xlm-roberta-base-tc-1
xlm-roberta-base-tc-2
xlm-roberta-base-tc-3
xlm-roberta-base-tc-4
xlm-roberta-base-tc-5


In [29]:
final_logits = 0
for saved_model in saved_models:
    model = AutoModelForSequenceClassification.from_pretrained(gdrive_path + 'saved/' + saved_model)
    tokenizer = AutoTokenizer.from_pretrained(gdrive_path + 'saved/' + saved_model)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    
    data_test = datasets.load_dataset('csv', data_files='/content/test_data.csv')['train']
    data_test = data_test.remove_columns('index')
    encoded_test = data_test.map(tokenize, batched=True, batch_size=None)
    encoded_test = encoded_test.remove_columns('title')

    training_args = TrainingArguments(output_dir='test_log/' + saved_model,
                                        per_device_eval_batch_size=batch_size,
                                        disable_tqdm=False)

    trainer = Trainer(model=model,
                args=training_args,
                tokenizer=tokenizer,
                data_collator=data_collator)

    preds_output = trainer.predict(encoded_test)

    final_logits += preds_output.predictions

Using custom data configuration default-5a3980b056bfd83d


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-5a3980b056bfd83d/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

...

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Prediction *****
  Num examples = 9131
  Batch size = 64


loading configuration file /content/drive/MyDrive/Data Science/알파코 딥러닝 부트캠프/프로젝트/뉴스 토픽 분류 AI 경진대회/saved/xlm-roberta-base-tc-5/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "/content/drive/MyDrive/Data Science/\u110b\u1161\u11af\u1111\u1161\u110f\u1169 \u1103\u1175\u11b8\u1105\u1165\u1102\u1175\u11bc \u1107\u116e\u1110\u1173\u110f\u1162\u11b7\u1111\u1173/\u1111\u1173\u1105\u1169\u110c\u1166\u11a8\u1110\u1173/\u1102\u1172\u1109\u1173 \u1110\u1169\u1111\u1175\u11a8 \u1107\u116e\u11ab\u1105\u1172 AI \u1100\u1167\u11bc\u110c\u1175\u11ab\u1103\u1162\u1112\u116c/saved/xlm-roberta-base-tc-5",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4"

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Prediction *****
  Num examples = 9131
  Batch size = 64


In [30]:
final_logits.argmax(1)

array([2, 3, 2, ..., 3, 2, 2])

In [31]:
sub = pd.read_csv("./sample_submission.csv")
sub['topic_idx'] = final_logits.argmax(1)
sub.to_csv('./final_submission.csv', index=False)

In [32]:
sub

,index,topic_idx
0,45654,2
1,45655,3
2,45656,2
3,45657,0
4,45658,3
...,...,...
9126,54780,3
9127,54781,2
9128,54782,3
9129,54783,2


### Ensemble Example

In [ ]:
def ensemble():
    final_logit=0
    # args.max_len=33
    args.pt = 'monologg/kobert'
    _, logit1 = run_predict("./saved_models/fold3/kobert/0f_9e_0.8895_s.pth")
    _, logit2 = run_predict("./saved_models/fold3/kobert/1f_10e_0.8823_s.pth")
    _, logit3 = run_predict("./saved_models/fold3/kobert/2f_8e_0.8888_s.pth")
    _, logit4 = run_predict("./saved_models/fold3/kobert/3f_10e_0.8897_s.pth")
    _, logit5 = run_predict("./saved_models/fold3/kobert/4f_8e_0.8867_s.pth")
    final_logit += (logit1+logit2+logit3+logit4+logit5)/5
    
    #####################

    args.pt = 'klue/roberta-base'
    _, logit1 = run_predict("./saved_models/fold3/roberta-base/0f_5e_0.8920_s.pth")
    _, logit2 = run_predict("./saved_models/fold3/roberta-base/1f_4e_0.8879_s.pth")
    _, logit3 = run_predict("./saved_models/fold3/roberta-base/2f_5e_0.8889_s.pth")
    _, logit4 = run_predict("./saved_models/fold3/roberta-base/3f_4e_0.8951_s.pth")
    _, logit5 = run_predict("./saved_models/fold3/roberta-base/4f_4e_0.8887_s.pth")

    final_logit += (logit1+logit2+logit3+logit4+logit5)/5

    #####################
    args.pt = 'klue/roberta-small'
    preds1, logit1 = run_predict("./saved_models/fold3/roberta-small/0f_8e_0.8900_s.pth")
    preds2, logit2 = run_predict("./saved_models/fold3/roberta-small/1f_9e_0.8813_s.pth")
    preds3, logit3 = run_predict("./saved_models/fold3/roberta-small/2f_7e_0.8884_s.pth")
    preds4, logit4 = run_predict("./saved_models/fold3/roberta-small/3f_3e_0.8958_s.pth")
    preds5, logit5 = run_predict("./saved_models/fold3/roberta-small/4f_4e_0.8881_s.pth") # 8884 가능
    final_logit += (logit1+logit2+logit3+logit4+logit5)/5
    #####################

    args.pt = 'bert-base-multilingual-uncased'
    preds1, logit1 = run_predict("./saved_models/fold3/bert-base-multilingual-uncased/0f_5e_0.8624_s.pth")
    preds2, logit2 = run_predict("./saved_models/fold3/bert-base-multilingual-uncased/1f_8e_0.8573_s.pth")
    preds3, logit3 = run_predict("./saved_models/fold3/bert-base-multilingual-uncased/2f_9e_0.8674_s.pth")
    preds4, logit4 = run_predict("./saved_models/fold3/bert-base-multilingual-uncased/3f_8e_0.8649_s.pth")
    preds5, logit5 = run_predict("./saved_models/fold3/bert-base-multilingual-uncased/4f_9e_0.8673_s.pth")
    final_logit += (logit1+logit2+logit3+logit4+logit5)/5
    #####################
    args.pt = 'klue/roberta-large'
    preds1, logit1 = run_predict("./saved_models/fold3/klue-roberta-large/0f_2e_0.8905_s.pth")
    preds2, logit2 = run_predict("./saved_models/fold3/klue-roberta-large/1f_4e_0.8897_s.pth")
    preds3, logit3 = run_predict("./saved_models/fold3/klue-roberta-large/2f_3e_0.8887_s.pth")
    preds4, logit4 = run_predict("./saved_models/fold3/klue-roberta-large/3f_3e_0.8949_s.pth")
    preds5, logit5 = run_predict("./saved_models/fold3/klue-roberta-large/4f_2e_0.8939_s.pth")
    final_logit += (logit1+logit2+logit3+logit4+logit5)/5
    #####################
    args.pt = 'xlm-roberta-large'
    preds1, logit1 = run_predict("./saved_models/fold3/xlm-roberta-large_radam/0f_6e_0.8928_s.pth")
    preds2, logit2 = run_predict("./saved_models/fold3/xlm-roberta-large_radam/1f_5e_0.8850_s.pth")
    preds3, logit3 = run_predict("./saved_models/fold3/xlm-roberta-large_radam/2f_5e_0.8891_s.pth")
    preds4, logit4 = run_predict("./saved_models/fold3/xlm-roberta-large_radam/3f_8e_0.8938_s.pth")
    preds5, logit5 = run_predict("./saved_models/fold3/xlm-roberta-large_radam/4f_6e_0.8911_s.pth")
    final_logit += (logit1+logit2+logit3+logit4+logit5)/5
    #####################
    args.max_len=28
    args.pt = 'klue/roberta-large'
    preds1, logit1 = run_predict("./saved_models/fold3/klue-roberta-large_28/0f_6e_0.8912_s.pth")
    preds2, logit2 = run_predict("./saved_models/fold3//klue-roberta-large_28/1f_3e_0.8891_s.pth")
    preds3, logit3 = run_predict("./saved_models/fold3//klue-roberta-large_28/2f_5e_0.8891_s.pth")
    preds4, logit4 = run_predict("./saved_models/fold3//klue-roberta-large_28/3f_4e_0.8961_s.pth")
    preds5, logit5 = run_predict("./saved_models/fold3//klue-roberta-large_28/4f_2e_0.8938_s.pth")
    final_logit += (logit1+logit2+logit3+logit4+logit5)/5
    
    return final_logit

### Redundants

In [ ]:
# notebook_login()

### Download Models & Tokenizers

In [ ]:
# models = ['klue/roberta-base', 'xlm-roberta-base', 'bert-base-multilingual-uncased']
# for adrs in models:
#     model = AutoModelForSequenceClassification.from_pretrained(adrs, num_labels=7)
#     tokenizer = AutoTokenizer.from_pretrained(adrs)

#     adrs.replace('/', '-')

#     tokenizer.save_pretrained(adrs)
#     model.save_pretrained(adrs)

### Data Load & K-Fold Interator Instantiation

In [ ]:
!gdown -q 1K0v7liLb4ls5NMZrvrsEFQCfGqGduBSN

In [ ]:
# df = pd.read_csv('/content/augmented_dataset.csv', index_col=False)
# df = df.drop(['Unnamed: 0'], axis=1)
# df.reset_index(inplace=True)
# X, y = df['title'], df['topic_idx']
# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=100)
# splitgen = skf.split(X, y)

### Model Test

In [ ]:
target_path = '/content/saved/xlm-roberta-base-tc-1'

model = AutoModelForSequenceClassification.from_pretrained(target_path)
tokenizer = AutoTokenizer.from_pretrained(target_path)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

data_test = datasets.load_dataset('csv', data_files='/content/test_data.csv')['train']
data_test = data_test.remove_columns('index')
encoded_test = data_test.map(tokenize, batched=True, batch_size=None)
encoded_test = encoded_test.remove_columns('title')

training_args = TrainingArguments(output_dir='test_log/' + saved_model,
                                    per_device_eval_batch_size=batch_size,
                                    disable_tqdm=False)

trainer = Trainer(model=model,
            args=training_args,
            tokenizer=tokenizer,
            data_collator=data_collator)

preds_output = trainer.predict(encoded_test)
preds_output

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(target_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(target_path)

In [ ]:
text = "this is a test"
inputs = tokenizer(text, return_tensors="pt")

inputs = {k:v.to(device) for k,v in inputs.items()}
with torch.no_grad():
 outputs = model(**inputs)
print(outputs)


SequenceClassifierOutput(loss=None, logits=tensor([[ 3.5732, -0.2511,  0.9122,  0.0313, -0.8860, -2.1067, -1.3475]],
       device='cuda:0'), hidden_states=None, attentions=None)


In [ ]:
print(inputs)

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1]], device='cuda:0'),
 'input_ids': tensor([[   0,  903,   83,   10, 3034,    2]], device='cuda:0')}